<a href="https://colab.research.google.com/github/PuppalaKarthik/karthik_INFO5731_Spring2023/blob/main/Assignments/INFO5731_Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon.

(2) Collect the top 10000 User Reviews of a film recently in 2023 or 2022 (you can choose any film) from IMDB.

(3) Collect all the reviews of the top 1000 most popular software from [G2](https://www.g2.com/) or [Capterra](https://www.capterra.com/)

(4) Collect the abstracts of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from [Semantic Scholar](https://www.semanticscholar.org).

(5) Collect all the information of the 904 narrators in the [Densho Digital Repository](https://ddr.densho.org/narrators/).

(6) Collect the top 10000 tweets by using a hashtag (you can use any hashtag) from Twitter. 


In [ ]:
!conda install selenium --yes

Solving environment: done

## Package Plan ##

  environment location: /Users/karthikpuppala/opt/anaconda3/envs/nlp

  added / updated specs:
    - selenium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    selenium-3.141.0           |py39h1a28f6b_1000         813 KB
    ------------------------------------------------------------
                                           Total:         813 KB

The following NEW packages will be INSTALLED:

  selenium           pkgs/main/osx-arm64::selenium-3.141.0-py39h1a28f6b_1000 

The following packages will be UPDATED:

  ca-certificates                     2022.07.19-hca03da5_0 --> 2023.01.10-hca03da5_0 
  certifi                          2022.9.24-py39hca03da5_0 --> 2022.12.7-py39hca03da5_0 
  openssl                                 1.1.1q-h1a28f6b_0 --> 1.1.1t-h1a28f6b_0 



                                                                       

In [ ]:
# Write your code here

import pandas as pd

import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By


In [ ]:
# Scraping product: Apple 2021 MacBook

root_url = "https://www.amazon.com/"
product_url = "https://www.amazon.com/Apple-MacBook-16-inch-10%E2%80%91core-16%E2%80%91core/product-reviews/B09JQKBQSB/ref=cm_cr_arp_d_viewopt_fmt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=1&formatType=all_formats"


In [ ]:
# Initiating Chrome or Edge Webservice

# Change mypath to the ms edge driver in your machine
mypath = "/Users/karthikpuppala/Downloads/edgedriver_mac64_m1/msedgedriver"

desired_cap = {} # Extra code to support MAC OS

#s = Service(mypath)
driver = webdriver.Edge(executable_path= mypath, capabilities=desired_cap)



In [ ]:
 def get_next_page_url(soup):
    # this will return the next page URL
    pages = soup.find('ul', class_='a-pagination')
    if not pages.find('li', class_='a-disabled a-last'):
        next_url = root_url + str(pages.find('li', class_='a-last').find('a')['href'])
        return next_url
    else:
        return


# def get_next_page_url(soup):
#     # this will return the next page URL
#     pages = soup.find('ul', class_='a-pagination')
#     if pages.find('li', class_='a-last'):
#         next_url = root_url + str(pages.find('li', class_='a-last').find('a')['href'])
#         return next_url
#     else:
#         return


In [ ]:
def get_page_soup_data(driver, url):

    # Change url as per requirement
    driver.get(url)    

    all_elements = driver.find_element(By.XPATH,"/html")

    #all_elements = driver.find_element(By.XPATH,"//main[@id='4sk8k']")
    html_doc = all_elements.get_attribute('outerHTML')

    soup = BeautifulSoup(html_doc,"html.parser")
    return soup


In [ ]:
def scrape_data(driver, url):
    
    # Creating a dictionary to store the data
    data = {
        'name': [],
        'rating': [],
        'review_title': [],
        'review_date': [],
        'review_body': [],
        'product_style': [],
        'product_capacity': [],
        'product_color': []
    }
    
    # Iterating only when next page is available
    while url:
        # getting soup data of the page
        soup = get_page_soup_data(driver, url)

        # Getting all reviews list in a page
        reviews_list = soup.find_all('div', class_='a-section review aok-relative')        

        # Iterating all reviews in a page
        for review in reviews_list:
            try:
                name = review.find('span', class_='a-profile-name').text.strip()
                rating = review.find('span', class_='a-icon-alt').text.strip()
                review_title = review.find('a', {'data-hook' :'review-title'}).find('span').text
                review_date = review.find('span', {'data-hook':'review-date'}).text
                review_body = review.find('span', {'data-hook':'review-body'}).find('span').text

                product_style = review.find('a', {'data-hook':'format-strip'}).text.split('Capacity')[0].split(':')[1].strip()
                product_capacity = review.find('a', {'data-hook':'format-strip'}).text.split(':')[2][:-5].strip()    
                product_color = review.find('a', {'data-hook':'format-strip'}).text.split(':')[3].strip()

    #             print(product_capacity)     
            except AttributeError:
                # handle errors due to missing or incomplete data
                print('\n------------------------')
                print('An error occurred while parsing the review:')
                print(f'Error present in url: {url}')
                print(f'Error present in review with title: {name}')
                print('------------------------\n')
                continue
            else:
                 # insering data into dict
                data['name'].append(name)
                data['rating'].append(rating)
                data['review_title'].append(review_title)
                data['review_date'].append(review_date)
                data['review_body'].append(review_body)
                data['product_style'].append(product_style)
                data['product_capacity'].append(product_capacity)
                data['product_color'].append(product_color)


        # getting next page url
        url = get_next_page_url(soup)

        print("Data collected till previous url: ", len(data['name']))
        print("Next URL: ", url)


#         # if next page then continuing else aborting
#         if not url:
#             driver.close()
#             break
    driver.close() # closing the Edge web driver once all pages scraping is done
    return data



In [ ]:
# Scrapes the data and stores all the data in dictionary variable data
data = scrape_data(driver, product_url)




------------------------
An error occurred while parsing the review:
Error present in url: https://www.amazon.com/Apple-MacBook-16-inch-10%E2%80%91core-16%E2%80%91core/product-reviews/B09JQKBQSB/ref=cm_cr_arp_d_viewopt_fmt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=1&formatType=all_formats
Error present in review with title: Nick Dreamz
------------------------

Data collected till previous url:  9
Next URL:  https://www.amazon.com//Apple-MacBook-16-inch-10%E2%80%91core-16%E2%80%91core/product-reviews/B09JQKBQSB/ref=cm_cr_arp_d_paging_btm_2?ie=UTF8&pageNumber=2&reviewerType=all_reviews&sortBy=recent&formatType=all_formats
Data collected till previous url:  19
Next URL:  https://www.amazon.com//Apple-MacBook-16-inch-10%E2%80%91core-16%E2%80%91core/product-reviews/B09JQKBQSB/ref=cm_cr_arp_d_paging_btm_3?ie=UTF8&pageNumber=3&reviewerType=all_reviews&sortBy=recent&formatType=all_formats

------------------------
An error occurred while parsing the review:
Error present in u

In [ ]:
df = pd.DataFrame(data)
df.shape

(227, 8)

In [ ]:
df.tail()

,name,rating,review_title,review_date,review_body,product_style,product_capacity,product_color
222,Joseph Thomas,5.0 out of 5 stars,Insanely fast.,Reviewed in the United States 🇺🇸 on October 31...,This laptop is unbelievably fast. Between the ...,Apple M1 Pro Chip,512 GB,Space Gray
223,Cade Prior,5.0 out of 5 stars,A video editors best friend.,Reviewed in the United States 🇺🇸 on October 30...,On day 3 of using this beast and wow. It’s ins...,Apple M1 Pro Chip,512 GB,Silver
224,Asha R.,5.0 out of 5 stars,This MacBook Pro Cured my depression!,Reviewed in the United States 🇺🇸 on October 29...,"Finally, this 2021 MacBook Pro cured my depres...",Apple M1 Pro Chip,512 GB,Silver
225,Anon,5.0 out of 5 stars,A laptop you can finally use on your lap,Reviewed in the United States 🇺🇸 on October 29...,Here are the temperature results for the new c...,Apple M1 Max Chip,1 TB,Space Gray
226,Aidan H,5.0 out of 5 stars,"Silent, beautiful mobile workstation",Reviewed in the United States 🇺🇸 on October 27...,This year's MacBook Pro with M1 Pro/Max merges...,Apple M1 Pro Chip,1 TB,Space Gray


# **Question 2**

(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming. 

(6) Lemmatization.

In [ ]:
# Write your code here

!conda install nltk -y

Solving environment: done

## Package Plan ##

  environment location: /Users/karthikpuppala/opt/anaconda3/envs/nlp

  added / updated specs:
    - nltk


The following packages will be UPDATED:

  ca-certificates    conda-forge::ca-certificates-2022.12.~ --> pkgs/main::ca-certificates-2023.01.10-hca03da5_0 

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi            conda-forge/noarch::certifi-2022.12.7~ --> pkgs/main/osx-arm64::certifi-2022.12.7-py39hca03da5_0 
  openssl            conda-forge::openssl-1.1.1t-h03a7124_0 --> pkgs/main::openssl-1.1.1t-h1a28f6b_0 




Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [ ]:
# import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/karthikpuppala/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/karthikpuppala/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/karthikpuppala/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
# Create a function to perform text cleaning operations
def clean_text(text):
    # Remove noise
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove numbers
    text = ''.join([i for i in text if not i.isdigit()])
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in nltk.word_tokenize(text) if word.lower() not in stop_words])
    # Lowercase the text
    text = text.lower()
    # Stem the text
    ps = PorterStemmer()
    text = ' '.join([ps.stem(word) for word in nltk.word_tokenize(text)])
    # Lemmatize the text
    wn = WordNetLemmatizer()
    text = ' '.join([wn.lemmatize(word) for word in nltk.word_tokenize(text)])
    return text

In [ ]:
# Creating a new dataframe to store cleaned data
df_clean = pd.DataFrame()

In [ ]:
# Apply the function to the 'review_title' & 'review_body' column and save the cleaned data to a new column

df_clean['review_title'] = df['review_title'].apply(lambda x: clean_text(x))
df_clean['review_body'] = df['review_body'].apply(lambda x: clean_text(x))


# df_clean['name'] = df['name'].apply(lambda x: clean_text(x))
# df_clean['rating'] = df['rating'].apply(lambda x: clean_text(x))
# df_clean['review_date'] = df['review_date'].apply(lambda x: clean_text(x))
# df_clean['product_style'] = df['product_style'].apply(lambda x: clean_text(x))
# df_clean['product_capacity'] = df['product_capacity'].apply(lambda x: clean_text(x))
# df_clean['product_color'] = df['product_color'].apply(lambda x: clean_text(x))

In [ ]:
# Save the updated dataframe to the CSV file
df_clean.to_csv('cleaned_data.csv', index=False)

In [ ]:
df_clean.head()

,review_title,review_body
0,amaz laptop,best laptop ever
1,price brand,excel peoduct realli satisfi
2,power macbook,upgrad inch macbook pro touchbar thing great s...
3,go wrong choic macbook pro inch,love much great job appl
4,amaz perform best valu hard earn money,im live somewher east africa order beast softw...


# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [ ]:
# Write your code here


import nltk

# download necessary resources
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('treebank')
nltk.download('conll2000')
nltk.download('treebank')
nltk.download('dependency_treebank')

from nltk.parse import DependencyGraph, DependencyEvaluator
from nltk.parse import stanford
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tree import Tree

# tokenize the text
from nltk.tokenize import word_tokenize



[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/karthikpuppala/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/karthikpuppala/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/karthikpuppala/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/karthikpuppala/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/karthikpuppala/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/karthikpuppala/nltk_data...
[nltk_data]   Package words is already 

----

In [ ]:
!conda install spacy -y

Solving environment: done

# All requested packages already installed.



In [ ]:
import spacy


In [ ]:
# !python -m spacy download en_core_web_sm

!conda install -c conda-forge spacy-model-en_core_web_sm -y

Solving environment: done

## Package Plan ##

  environment location: /Users/karthikpuppala/opt/anaconda3/envs/nlp

  added / updated specs:
    - spacy-model-en_core_web_sm


The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    pkgs/main::ca-certificates-2023.01.10~ --> conda-forge::ca-certificates-2022.12.7-h4653dfc_0 
  certifi            pkgs/main/osx-arm64::certifi-2022.12.~ --> conda-forge/noarch::certifi-2022.12.7-pyhd8ed1ab_0 
  openssl              pkgs/main::openssl-1.1.1t-h1a28f6b_0 --> conda-forge::openssl-1.1.1t-h03a7124_0 




Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
def syntax_structure_analysis(text):
    # POS tagging
    doc = nlp(text)
    noun_count = 0
    verb_count = 0
    adj_count = 0
    adv_count = 0
    for token in doc:
        if token.pos_ == 'NOUN':
            noun_count += 1
        elif token.pos_ == 'VERB':
            verb_count += 1
        elif token.pos_ == 'ADJ':
            adj_count += 1
        elif token.pos_ == 'ADV':
            adv_count += 1
            
    print("POS tagging:")
    print('Noun count:', noun_count)
    print('Verb count:', verb_count)
    print('Adjective count:', adj_count)
    print('Adverb count:', adv_count)
    print()

    # Constituency parsing and dependency parsing
    for sentence in doc.sents:
        print('Sentence: ')
        print(sentence.text)
        print()
        cp = ''
        for chunk in sentence.noun_chunks:
            cp += str(chunk) + ' '
        print('Constituency Parsing Tree:', cp)
        print()
        dp = ''
        for token in sentence:
            dp += '(' + token.text + ', ' + token.dep_ + ', ' + token.head.text + ')\n'
        print('Dependency Parsing Tree:', dp)
        print()
    
    # Named entity recognition
    entity_count = {}
    for ent in doc.ents:
        if ent.label_ not in entity_count:
            entity_count[ent.label_] = 1
        else:
            entity_count[ent.label_] += 1
    
    print("Named entity recognition: ")
    print('Person count:', entity_count.get('PERSON', 0))
    print('Organization count:', entity_count.get('ORG', 0))
    print('Location count:', entity_count.get('GPE', 0))
    print('Product count:', entity_count.get('PRODUCT', 0))
    print('Date count:', entity_count.get('DATE', 0))
    print()


In [ ]:
# apply syntax and structure analysis to each review_title and review_body
for idx, row in df_clean.iterrows():
    
    print('Review Title:', end='\n\n')
    syntax_structure_analysis(row['review_title'])
    print("----------------------------------------")
    print('Review Body:', end='\n\n')
    syntax_structure_analysis(row['review_body'])
    print("----------------------------------------")
    print("----------------------------------------")
    print()

Review Title:

POS tagging:
Noun count: 1
Verb count: 0
Adjective count: 0
Adverb count: 0

Sentence: 
amaz laptop

Constituency Parsing Tree: 

Dependency Parsing Tree: (amaz, ROOT, amaz)
(laptop, npadvmod, amaz)


Named entity recognition: 
Person count: 0
Organization count: 0
Location count: 0
Product count: 0
Date count: 0

----------------------------------------
Review Body:

POS tagging:
Noun count: 1
Verb count: 0
Adjective count: 1
Adverb count: 1

Sentence: 
best laptop ever

Constituency Parsing Tree: best laptop 

Dependency Parsing Tree: (best, amod, laptop)
(laptop, ROOT, laptop)
(ever, advmod, laptop)


Named entity recognition: 
Person count: 0
Organization count: 0
Location count: 0
Product count: 0
Date count: 0

----------------------------------------
----------------------------------------

Review Title:

POS tagging:
Noun count: 2
Verb count: 0
Adjective count: 0
Adverb count: 0

Sentence: 
price brand

Constituency Parsing Tree: price brand 

Dependency Parsing

**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):** 

Constituency parsing tree and dependency parsing tree are two types of syntax trees that represent the structure of a sentence. Here are some key points that I understood:

**Constituency Parsing Tree:**

A constituency parsing tree represents the hierarchical structure of a sentence by breaking it down into smaller constituents, such as noun phrases and verb phrases.
Each constituent is represented as a node in the tree, and the relations between them are shown by the edges connecting them.
Constituency parsing tree helps in identifying the phrases and sub-phrases present in the sentence.

**Dependency Parsing Tree:**

A dependency parsing tree represents the grammatical relationships between the words in a sentence.
Each word is represented as a node in the tree, and the relations between them are shown by directed edges.
The tree starts from the root node and then branches out to its dependencies, showing which words modify or are modified by others in the sentence.
